## Generates the poule sheets for Judo+ tournaments  
#### Work by [Gonçalo Sousa](https://github.com/Sousa1909) for [AJDS](https://github.com/AJD-Santarem)  

---

In [4]:
import pandas as pd
from openpyxl import load_workbook
import os
import random

# Create Input Folder if it doesn't exist
if not os.path.exists("Input"):
    os.makedirs("Input")
    print("Input folder does not exist. Generating...")

Table Manipulation

In [5]:
# Read the Excel file (Masculine and Feminine sheets) without skipping rows
df_masc = pd.read_excel('Input/judo+_test.xlsx', sheet_name='MASC')
df_fem = pd.read_excel('Input/judo+_test.xlsx', sheet_name='FEM')

# Names for the output PDFs
tournament_code = "judo_plus_test"

# Function to process the data (drop the first row, set column names, and sort)
def process_table(df):
    # Drop the first row ( TITLE )
    df = df.drop(index=[0])

    # Set new column names
    df.columns = ['NAME', 'WEIGHT', 'CLUB']

    # Sort by CLUB (ascending) and WEIGHT (ascending)
    df = df.sort_values(by=['WEIGHT'], ascending=[True])

    return df

# Process both tables
df_masc = process_table(df_masc)
df_fem = process_table(df_fem)

Athlete Grouping and Draw

In [6]:
# Function to check if a group has an athlete from the same club
def has_same_club(group, club):
    return any(athlete['CLUB'] == club for athlete in group)

# Function to split a list into groups of 5, 4, or 3, ideally
def divide_into_groups(athletes, column_order, target_group_sizes=[4, 3]):
    random.shuffle(athletes)  # Shuffle athletes randomly
    result_groups = []
    
    # Split into groups of the preferred sizes (4 or 3)
    i = 0
    while i < len(athletes):
        remaining = len(athletes) - i
        
        # Try to form a group of 4 or 3 based on the remaining number of athletes
        for size in target_group_sizes:
            if remaining >= size or remaining == 2:  # If we're down to 2 athletes, group them as 2
                group = []
                for athlete in athletes[i:i + size]:
                    # Check if the athlete's club is already in the group, and avoid if possible
                    if has_same_club(group, athlete['CLUB']):
                        # Try to add this athlete to a different group, but only if groups exist
                        added = False
                        if result_groups:
                            for prev_group in result_groups:
                                if len(prev_group) < 5 and not has_same_club(prev_group.to_dict(orient='records'), athlete['CLUB']):
                                    prev_group = pd.concat([prev_group, pd.DataFrame([athlete])])
                                    added = True
                                    break
                        if not added:
                            group.append(athlete)
                    else:
                        group.append(athlete)
                group_df = pd.DataFrame(group)  # Convert to DataFrame
                result_groups.append(group_df[column_order])  # Reorder columns consistently
                i += size
                break
        else:
            # Handle the case where we have one athlete left
            group = athletes[i:]
            group_df = pd.DataFrame(group)
            # Save this group as a "lone" athlete
            result_groups.append(group_df[column_order])
            break

    # Handle the case of a lone athlete by merging them with another group
    if len(result_groups[-1]) == 1:
        lone_athlete = result_groups.pop()  # Remove the lone athlete's group
        for group in result_groups:
            if len(group) < 5:  # Add them to a group with less than 5 members
                result_groups[result_groups.index(group)] = pd.concat([group, lone_athlete], ignore_index=True)
                break

    return result_groups

# Function to group athletes by 4kg weight ranges
def group_by_weight_and_divide(df, weight_range=4.0):
    column_order=['NAME', 'WEIGHT', 'CLUB']
    df = df.sort_values(by='WEIGHT').reset_index(drop=True)  # Sort by weight
    groups = []
    current_group = []
    current_limit = df.iloc[0]['WEIGHT'] + weight_range  # Set initial weight range limit
    
    for _, row in df.iterrows():
        if row['WEIGHT'] <= current_limit:
            current_group.append(row)
        else:
            groups.append(current_group)  # Save the group
            current_group = [row]  # Start a new group
            current_limit = row['WEIGHT'] + weight_range  # Set new weight limit for the next group
    
    groups.append(current_group)  # Append the last group

    # Divide each weight group into subgroups of 5, 4, or 3
    final_groups = []
    for group in groups:
        athlete_list = [row for row in group]
        divided_groups = divide_into_groups(athlete_list, column_order=column_order)
        final_groups.extend(divided_groups)

    return final_groups

def save_grouped_athletes_to_file(groups, gender):

    # Create the folder if it doesn't exist
    output_folder = "Output"
    os.makedirs(output_folder, exist_ok=True)
    
    output_text_file = f'Output/.grouped_athletes_{gender}.txt'
    
    with open(output_text_file, 'w') as f:
        for i, group in enumerate(groups):
            f.write(f"Group {i + 1}:\n")  # Write the group number
            for athlete in group.itertuples(index=False):  # Iterate through the athletes in the group
                f.write(f"Name: {athlete.NAME}, Club: {athlete.CLUB}, Weight: {athlete.WEIGHT}\n")  # Write athlete details
            f.write("\n")  # Add a blank line between groups

    print(f"Grouped athletes for {gender} have been saved to {output_text_file}.")

groups_fem = group_by_weight_and_divide(df_fem)
save_grouped_athletes_to_file(groups_fem, 'fem')
groups_mas = group_by_weight_and_divide(df_masc)
save_grouped_athletes_to_file(groups_mas, 'mas')

Grouped athletes for fem have been saved to Output/.grouped_athletes_fem.txt.
Grouped athletes for mas have been saved to Output/.grouped_athletes_mas.txt.


In [7]:
best_of_3 = "Templates/BO_3.xlsx"
poule_of_5 ="Templates/Poule_5.xlsx"

# Load the appropriate template based on group size
def load_template(group_size, template_file_5, template_file_2):
    if group_size == 2:
        return True, load_workbook(template_file_2)
    else:
        return False,load_workbook(template_file_5)

# Populate the table with the group information
def populate_group_table(group, template_file_5, template_file_2, output_file, group_number):
    # Load the correct template based on the group size
    bo3, workbook = load_template(len(group), template_file_5, template_file_2)
    sheet = workbook.active

    # Fill in athlete data
    if bo3:
        for i, athlete in enumerate(group.itertuples(), start=6):
            sheet[f'C{i + 2}'] = athlete.NAME
            sheet[f'D{i + 2}'] = athlete.CLUB
        sheet[f'F{13}'] = group_number
    else:
        for i, athlete in enumerate(group.itertuples(), start=7):
            sheet[f'B{i + 2}'] = athlete.NAME
            sheet[f'C{i + 2}'] = athlete.CLUB
        sheet[f'L{17}'] = group_number

    # Save the updated file to the designated folder
    output_filename = os.path.join("Output", output_file.format(group_number))
    workbook.save(output_filename)
    #print(f"Group {group_number} saved to {output_filename}")

# Function to process each group and generate tables
def create_tables_for_groups(groups, template_file_5, template_file_2, output_file_pattern):
    for idx, group in enumerate(groups, start=1):
        populate_group_table(group, template_file_5, template_file_2, output_file_pattern, idx)

create_tables_for_groups(groups_mas, poule_of_5, best_of_3, 'Group_M_{}.xlsx')
create_tables_for_groups(groups_fem, poule_of_5, best_of_3, 'Group_F_{}.xlsx')

NameError: name 'output_folder' is not defined